In [1]:
import os.path

import tensorflow as tf
from tensorflow.contrib import slim

import dataset_interval_reader
from shared_examples_queue import SharedExamplesQueue
from models import SequenceAndDnaseClassifier
from trainers import ClassiferTrainer

In [2]:
NUM_TASKS = 6

DATA_DIR = 'test-data'
LOG_DIR = 'test-logs'

INTERVALS_FILE = os.path.join(DATA_DIR, 'intervals_file.json')
INPUTS_FILE = os.path.join(DATA_DIR, 'inputs_file.json')

In [3]:
! rm -rf {LOG_DIR}/*

In [4]:
readers, task_names = dataset_interval_reader.get_readers_and_tasks(INPUTS_FILE, INTERVALS_FILE)
shared_queue = SharedExamplesQueue(readers, task_names, batch_size=32)

In [5]:
model = SequenceAndDnaseClassifier(num_tasks=NUM_TASKS)

In [6]:
trainer = ClassiferTrainer(model, epoch_size=5000)

In [7]:
opt = trainer.train(shared_queue, LOG_DIR)

seq_preds shape (32, 1, 928, 25)
dnase_preds shape (32, 1, 928, 25)
concat shape (32, 2, 928, 25)
after combined conv layers shape (32, 1, 904, 55)
INFO:tensorflow:Summary name sequence_conv1/weights:gradient is illegal; using sequence_conv1/weights_gradient instead.
INFO:tensorflow:Summary name sequence_conv1/weights:gradient_norm is illegal; using sequence_conv1/weights_gradient_norm instead.
INFO:tensorflow:Summary name sequence_conv1/biases:gradient is illegal; using sequence_conv1/biases_gradient instead.
INFO:tensorflow:Summary name sequence_conv1/biases:gradient_norm is illegal; using sequence_conv1/biases_gradient_norm instead.
INFO:tensorflow:Summary name sequence_conv2/weights:gradient is illegal; using sequence_conv2/weights_gradient instead.
INFO:tensorflow:Summary name sequence_conv2/weights:gradient_norm is illegal; using sequence_conv2/weights_gradient_norm instead.
INFO:tensorflow:Summary name sequence_conv2/biases:gradient is illegal; using sequence_conv2/biases_gradie

KeyboardInterrupt: 

In [9]:
opt.compute_gradients?

In [ ]:
s = tf.InteractiveSession()
s.run(tf.global_variables_initializer())
s.run(tf.local_variables_initializer())
_ = tf.train.start_queue_runners(s)

In [ ]:
labels = shared_queue.outputs['labels']#.eval()
dataset_idxs = shared_queue.outputs['dataset-index']#.eval()

fc_out = tf.random_normal([32, NUM_TASKS])
logits = fc_out

In [ ]:
labels[dataset_idxs == 1,:].eval()

In [ ]:
tf.nn.softmax_cross_entropy_with_logits(fc_out, )

In [11]:
from tensorflow.contrib import slim

In [13]:
slim.get_model_variables()

In [14]:
v = slim.get_model_variables()[0]

In [20]:
opt.get_slot?

In [ ]:
for dset_idx in range(3):
    print(tf.boolean_mask(labels, tf.equal(dataset_idxs, dset_idx)).eval().shape)

In [ ]:
tf.equal(labels, -1)

In [ ]:
ret = tf.ones_like(labels)
tf.where(tf.equal(labels, -1), tf.zeros_like(labels), tf.ones_like())

In [ ]:
sigmoid_xentropy_loss = slim.losses.sigmoid_cross_entropy(logits, labels)

In [ ]:
sigmoid_xentropy_loss

In [ ]:
labels.dtype

In [ ]:
tf.zeros_like?

In [9]:
shared_queue.outputs.keys()

['intervals/start',
 'labels',
 'data/dnase_data_dir',
 'dataset',
 'intervals/chrom',
 'dataset-index',
 'data/genome_data_dir',
 'intervals/end']

In [ ]:
shared_queue.